# 10 Minute Tour of PyTestLab

Welcome to PyTestLab! This short tour will guide you through the basic setup and usage of PyTestLab to interact with your lab instruments.

## 1. Installation

PyTestLab can be installed using pip. If you haven't already, open your terminal and run:

In [ ]:
!pip install pytestlab

Make sure you have the necessary VISA libraries installed for your operating system if you plan to connect to real instruments (e.g., NI-VISA, Keysight Connection Expert).

## 2. Importing PyTestLab and Connecting to an Instrument

PyTestLab uses an `AutoInstrument` class to easily load instrument configurations and connect. Configurations can be loaded from a CDN, local files, or a dictionary.

In [ ]:
import asyncio
from pytestlab.instruments import AutoInstrument
from pytestlab.common.enums import DMMFunction # For DMM example
from pytestlab.experiments.database import MeasurementDatabase # For DB example
from pytestlab.experiments.results import MeasurementResult # For DB example

### Connecting to a Simulated Instrument

For this tour, we'll start with a simulated instrument, which doesn't require any physical hardware.

In [ ]:
async def connect_simulated_scope():
    # Load a configuration for a Keysight DSOX1204G oscilloscope
    # 'simulate=True' tells AutoInstrument to use the simulation backend.
    scope_sim = AutoInstrument.from_config("keysight/DSOX1204G", simulate=True)
    
    # Connect to the backend (important step for async instruments)
    await scope_sim.connect_backend()
    
    print(f"Successfully connected to simulated: {await scope_sim.id()}")
    return scope_sim

async def main_sim():
    scope = await connect_simulated_scope()
    # You can now interact with 'scope'
    await scope.close() # Close the connection when done

# To run in a Jupyter Notebook, you can use asyncio.run() or get_event_loop().run_until_complete()
# For simplicity in this example, we'll show how you might call it.
# If you are in a Jupyter environment with an existing event loop, you might need:
# await main_sim() 
# Or, if no loop is running:
if __name__ == "__main__":
    asyncio.run(main_sim())

### Connecting to a Real Instrument (Example)

To connect to a real instrument, you would set `simulate=False` and provide the VISA address. The `config_source` can be a path to a local YAML file or a CDN identifier.

In [ ]:
# This is an example and will likely fail if you don't have this instrument at this address.
async def connect_real_psu():
    try:
        # Example: Connecting to a Keysight E36313A Power Supply
        # Replace with your instrument's actual VISA address and config identifier.
        psu_real = AutoInstrument.from_config(
            config_source="keysight/E36313A", # CDN or local file path
            resource_name="TCPIP0::YOUR_PSU_IP_ADDRESS::INSTR", # Example VISA address
            simulate=False
        )
        await psu_real.connect_backend()
        print(f"Successfully connected to: {await psu_real.id()}")
        return psu_real
    except Exception as e:
        print(f"Failed to connect to real PSU: {e}")
        return None

async def main_real_example():
    psu = await connect_real_psu()
    if psu:
        # Interact with the PSU
        await psu.close()

# if __name__ == "__main__":
#     asyncio.run(main_real_example())

## 3. Basic Operations using New Facades

PyTestLab now provides intuitive facade objects for common instrument operations, allowing for a chained, more readable syntax. These facade methods are `async`.

### Power Supply (PSU) Facade Example

In [ ]:
async def psu_facade_example():
    # Using a simulated PSU for this example
    psu = AutoInstrument.from_config("keysight/E36311A", simulate=True) # Example PSU config
    await psu.connect_backend()
    print(f"PSU ID: {await psu.id()}")

    try:
        # Using the PSUChannelFacade
        # Configure channel 1: set voltage to 3.3V, current limit to 0.5A, then turn it on.
        await psu.channel(1).set(voltage=3.3, current_limit=0.5).on()
        print(f"PSU Channel 1: Voltage set to 3.3V, Current Limit to 0.5A, Output ON")

        # Read back the voltage (assuming the facade has get_voltage)
        # Note: The example PSUChannelFacade.set returns self, for get_voltage you'd call it separately.
        ch1_voltage = await psu.channel(1).get_voltage()
        ch1_current = await psu.channel(1).get_current()
        ch1_state = await psu.channel(1).get_output_state()
        print(f"PSU Channel 1 Readback: Voltage={ch1_voltage}V, Current={ch1_current}A, State={'ON' if ch1_state else 'OFF'}")

        # Turn channel 1 off
        await psu.channel(1).off()
        print(f"PSU Channel 1: Output OFF")
        ch1_state_after_off = await psu.channel(1).get_output_state()
        print(f"PSU Channel 1 Readback after off: State={'ON' if ch1_state_after_off else 'OFF'}")

    except Exception as e:
        print(f"An error occurred with PSU facade: {e}")
    finally:
        await psu.close()

# To run in Jupyter:
# await psu_facade_example()
# Or if no loop is running:
if __name__ == "__main__":
    asyncio.run(psu_facade_example())

### Oscilloscope Facade Example

In [ ]:
from pytestlab.common.enums import TriggerSlope # Ensure TriggerSlope is imported

async def oscilloscope_facade_example():
    scope = AutoInstrument.from_config("keysight/DSOX1204G", simulate=True)
    await scope.connect_backend()
    print(f"Scope ID: {await scope.id()}")

    try:
        # Channel Facade: Configure channel 1
        await scope.channel(1).setup(scale=0.5, position=0.0, coupling='DC').enable()
        print("Scope Channel 1: Scale 0.5V/div, Position 0.0s, Coupling DC, Enabled")

        # Trigger Facade: Setup edge trigger on channel 1
        await scope.trigger.setup_edge(source='CH1', level=1.0, slope=TriggerSlope.POSITIVE)
        print("Scope Trigger: Edge trigger on CH1, Level 1.0V, Slope POSITIVE")

        # Acquisition Facade: Start acquisition, wait for trigger, get waveform
        print("Scope Acquisition: Starting...")
        waveform_data = await scope.acquisition.start().wait_for_trigger(timeout=5).get_waveform(1)
        # Note: wait_for_trigger in the example implementation might be basic.
        # A real scenario might involve more complex trigger status polling.
        
        print(f"Scope Acquisition: Waveform data for channel 1 acquired (first 5 points):\n{waveform_data.values.head(5)}")
        
    except Exception as e:
        print(f"An error occurred with Oscilloscope facade: {e}")
    finally:
        await scope.close()

# To run in Jupyter:
# await oscilloscope_facade_example()
# Or if no loop is running:
if __name__ == "__main__":
    asyncio.run(oscilloscope_facade_example())

### Waveform Generator (WG) Facade Example

In [ ]:
async def wg_facade_example():
    wg = AutoInstrument.from_config("keysight/EDU33212A", simulate=True) # Example WG config
    await wg.connect_backend()
    print(f"Waveform Generator ID: {await wg.id()}")

    try:
        # Configure channel 1 to output a sine wave
        await wg.channel(1).setup_sine(frequency=1e3, amplitude=1.0, offset=0.0).enable()
        print("WG Channel 1: Sine wave, 1kHz, 1.0Vpp, 0V offset, Output ON")

        # Example: Get some configuration (actual methods depend on WGChannelFacade implementation)
        # current_func = await wg.channel(1).get_function_type() # Assuming such a method exists
        # current_freq = await wg.channel(1).get_frequency()     # Assuming such a method exists
        # print(f"WG Channel 1 Readback: Function={current_func}, Frequency={current_freq}Hz")
        
        # Disable output
        await wg.channel(1).disable()
        print("WG Channel 1: Output OFF")

    except Exception as e:
        print(f"An error occurred with Waveform Generator facade: {e}")
    finally:
        await wg.close()

# To run in Jupyter:
# await wg_facade_example()
# Or if no loop is running:
if __name__ == "__main__":
    asyncio.run(wg_facade_example())

## 4. Performing a Simple Measurement (DMM)

In [ ]:
async def dmm_measurement_example():
    dmm = AutoInstrument.from_config("keysight/EDU34450A", simulate=True) # Example DMM config
    await dmm.connect_backend()
    print(f"DMM ID: {await dmm.id()}")

    measurement_result = None
    try:
        # Measure DC Voltage
        # The DMM 'measure' method is async and takes a DMMFunction enum member.
        measurement_result = await dmm.measure(DMMFunction.DC_VOLTAGE, range_val="AUTO", resolution="DEF")
        print(f"DMM Measurement: {measurement_result.measurement_type} = {measurement_result.values} {measurement_result.units}")
        
    except Exception as e:
        print(f"An error occurred during DMM measurement: {e}")
    finally:
        await dmm.close()
    return measurement_result # Return for database example

# To run in Jupyter:
# dmm_result = await dmm_measurement_example()
# Or if no loop is running:
if __name__ == "__main__":
    dmm_result_for_db = asyncio.run(dmm_measurement_example())

## 5. Storing and Retrieving Results with MeasurementDatabase

In [ ]:
def database_example(measurement_to_store):
    if not measurement_to_store:
        print("No measurement to store. Skipping database example.")
        # Create a dummy measurement if needed for the example to run
        measurement_to_store = MeasurementResult(
            values=1.234, 
            instrument="SimulatedDMM", 
            units="V", 
            measurement_type="DC Voltage (Dummy)"
        )
        print(f"Using dummy measurement: {measurement_to_store}")

    # Initialize the database (creates 'my_lab_data.db' if it doesn't exist)
    with MeasurementDatabase("my_lab_data") as db:
        print(f"Database initialized at: {db.db_path}")
        
        # Store the measurement result
        # store_measurement is synchronous
        try:
            measurement_codename = db.store_measurement(
                codename=None, # Let the DB generate a codename
                measurement=measurement_to_store,
                notes="This is a test measurement from the 10-minute tour."
            )
            print(f"Measurement stored with codename: {measurement_codename}")

            # Retrieve the stored measurement
            retrieved_measurement = db.retrieve_measurement(measurement_codename)
            print(f"Retrieved Measurement ({retrieved_measurement.instrument}): {retrieved_measurement.values} {retrieved_measurement.units}")
            print(f"Timestamp: {datetime.fromtimestamp(retrieved_measurement.timestamp)}")

            # List recent measurements
            recent_measurements = db.list_measurements(limit=5)
            print(f"\nRecent measurement codenames: {recent_measurements}")

            # Search for measurements
            search_results = db.search_measurements(query="tour")
            print(f"\nSearch results for 'tour':")
            for res in search_results:
                print(f"  - {res['codename']}: {res['measurement_type']} from {res['instrument']}")
        except Exception as e:
            print(f"Database operation failed: {e}")

# if __name__ == "__main__":
    # This assumes dmm_result_for_db was populated by the previous cell's asyncio.run
    # In a real script, you'd manage the async execution and pass results properly.
    # For this notebook, we'll call it directly if the variable exists.
    # if 'dmm_result_for_db' in locals() and dmm_result_for_db:
    #     database_example(dmm_result_for_db)
    # else:
    #     print("DMM result not available for database example. Running with dummy data.")
    #     database_example(None) # Run with dummy data if DMM example didn't produce a result

# Simplified execution for notebook demonstration assuming previous cell ran
if __name__ == '__main__':
    # First, run the DMM example to get a result
    print("--- Running DMM Example First ---")
    dmm_res = asyncio.run(dmm_measurement_example())
    print("--- DMM Example Finished ---\n")
    
    print("--- Running Database Example ---")
    database_example(dmm_res)
    print("--- Database Example Finished ---")

## 6. Next Steps

This tour covered the basics of installing PyTestLab, connecting to instruments (simulated and real), using the new asynchronous facades for common operations, and storing/retrieving data.

From here, you can explore:
- Detailed documentation for specific instruments.
- Advanced features like experiment management and complex sweeps.
- Creating your own instrument configuration files.
- Integrating PyTestLab into your automated test scripts.